In [8]:
import pandas as pd
import geopandas as gpd

In [2]:
source_dir = '/mnt/c/Users/natra/Documents/Research/Iraq-post-conflict-rebel-governance'

In [4]:
iom_2022_file = source_dir + '/data/iom/2022_collapsed.csv'
iom_2022 = pd.read_csv(iom_2022_file)

In [7]:
iom_2022.head()

,LocationID,Governorate,District,Subdistrict,LocationType,PlaceType,LocationEn,isgov
0,2101002,Anbar,Al-Ka'im,Al-Obiadi,Peri-urban,Neighborhood,Al Aubaidi Al Qadima,1
1,2101033,Anbar,Al-Ka'im,Al-Obiadi,Peri-urban,Neighborhood,Tiwaan,1
2,2101032,Anbar,Al-Ka'im,Al-Obiadi,Rural,Village,Al Masharea,1
3,2101010,Anbar,Al-Ka'im,Al-Obiadi,Rural,Village,Al Zalla,1
4,2101034,Anbar,Al-Ka'im,Al-Obiadi,Rural,Village,Dghima village,1


In [14]:
print(len(iom_2022.Subdistrict.unique()))
print(len(iom_2022.District.unique()))
print(len(iom_2022.Governorate.unique()))

264
94
18


In [10]:
iraq_shp_adm3_loc = source_dir + '/data/geo_boundaries/irq-administrative-divisions-shapefiles/irq_admbnda_adm3_cso_20190603.shp'
iraq_shp_adm3 = gpd.read_file(iraq_shp_adm3_loc)

In [13]:
print(len(iraq_shp_adm3.ADM3_EN.unique()))
print(len(iraq_shp_adm3.ADM2_EN.unique()))
print(len(iraq_shp_adm3.ADM1_EN.unique()))

294
101
18


Governorates correspond to Adm 1 locations - 18 total

Districts - there are 105 in the overall IOM DTM data, but only 101 in Admin 2.

Subdistricts - correspond to Adm 3 locations.  

Location - according to IOM's definition, corresponds to the 4th official administrative division (https://iraqdtm.iom.int/MasterList#Methodology). However, the shapefiles for these adm 4 locations do not seem to be available (as described in the Metadata page for the Iraq Populated Places database OCHA created: https://data.humdata.org/dataset/iraq-populated-places-2021, where they say "If, in the future, administrative level 4 boundaries are obtained, these P-codes may need to be regenerated.")

In [16]:
# testing how easy it is to join IOM with Adm 3 names
iom_subdistricts = pd.DataFrame({'iom_subd':iom_2022.Subdistrict.unique()})
iraq_adm3_names = pd.DataFrame({'adm3':iraq_shp_adm3.ADM3_EN.unique()})

In [20]:
join_names = pd.merge(iom_subdistricts, iraq_adm3_names, left_on='iom_subd', right_on='adm3',how='outer')
print(join_names.shape)
# Adm 3 names that don't immediately match to IOM Subdistrict names
print(join_names[join_names['iom_subd'].isna()].shape)
# IOM Subdistrict names that don't immediately match to Adm 3 names
print(join_names[join_names['adm3'].isna()].shape)

(449, 2)
(185, 2)
(155, 2)


In [22]:
iom_subd_tocorrect = join_names[join_names['adm3'].isna()]['iom_subd']
adm3_tocorrect = join_names[join_names['iom_subd'].isna()]['adm3']

In [33]:
names_tomatch = pd.DataFrame({'iom_subdistict':list(sorted(iom_subd_tocorrect))})
names_tomatch = names_tomatch.join(pd.DataFrame({'adm3_en':list(sorted(adm3_tocorrect))}),how='outer')
print(names_tomatch.shape)
names_tomatch.head()

(185, 2)


,iom_subdistict,adm3_en
0,9 Nissan,Abi Gharaq
1,AL-Hurr,Abu Dalf
2,AL-Khayrat,Abu Ghraib
3,Abaiji,Agjalare
4,Abo Sayda,Al-Abbasi


In [35]:
names_tomatch_file = source_dir + 'adm3_names_tomatch.csv'
names_tomatch.to_csv(names_tomatch_file)